In [ ]:
from utils import *
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Access the environment variables
# HELIUS_API = os.getenv('HELIUS_API')
# DATABASE_URL = os.getenv('DATABASE_URL')
# TABLE_NAME = os.getenv('TABLE_NAME')
WALLET = os.getenv('WALLET')


In [ ]:
# fetch all txs from db
txs = select_from_db(WALLET)

# transform and save to db (everything)
# TODO: transform just new txs.
# 3QHi4kSpHTVjAeheTJ79F6eoTiaPbHfbpDkmQT1TCHuYPVTJJXYBZh2HnLvbUXjzDqCWv7i9F9tUk9e56wC6YgoY
# 2QuyzrSSjs6ntMLBwGjUViiKvt688X6HytQprGkAWBsMySM6cJUqKoCsjauHYkv53jpmbML68UhyFnF8A3G6ns2K
df = transform_to_dataframe(WALLET, txs)
df = df.sort_values(by="timestamp").reset_index(drop=True)

In [ ]:
pd.set_option('display.max_colwidth', 15)

In [ ]:
def update_sol_spent_received(df):
# Iterate over each row and update the sol_spent and sol_received columns
    df['sol_spent'] = 0.0
    df['sol_received'] = 0.0

    for i in range(len(df)):
        row = df.iloc[i]
        typeop = row['typeop']
        mint = row['mint']
        source_amount = row['source_amount']

        # Update sol_spent when typeop == 'BUY'
        if typeop == 'BUY':
            previous_rows = df[(df['typeop'] == 'BUY') & (df['mint'] == mint) & (df.index < i)]
            if not previous_rows.empty:
                last_row_index = previous_rows.index[-1]
                sol_spent_previous = df.at[last_row_index, 'sol_spent']
                df.at[i, 'sol_spent'] = sol_spent_previous + source_amount
            else:
                df.at[i, 'sol_spent'] = source_amount

        # Update sol_received when typeop == 'SELL'
        if typeop == 'SELL':
            previous_rows = df[(df['typeop'] == 'SELL') & (df['mint'] == mint) & (df.index < i)]
            if not previous_rows.empty:
                last_row_index = previous_rows.index[-1]
                sol_received_previous = df.at[last_row_index, 'sol_received']
                df.at[i, 'sol_received'] = sol_received_previous + source_amount
            else:
                df.at[i, 'sol_received'] = source_amount
    return df




df = update_sol_spent_received(df)


In [ ]:
def calculate_amounts(df):

    # Initialize the previous amount and new amount columns
    df['previous amount'] = 0.0
    df['new amount'] = 0.0

    # Iterate over each row and update the previous amount and new amount columns
    for i in range(len(df)):
        row = df.iloc[i]
        mint = row['mint']
        token_amount = row['token_amount']

        # Find the previous row with the same mint
        previous_rows = df[(df['mint'] == mint) & (df.index < i)]
        if not previous_rows.empty:
            last_row_index = previous_rows.index[-1]
            previous_amount = df.at[last_row_index, 'new amount']
        else:
            previous_amount = 0.0

        # Set the previous amount
        df.at[i, 'previous amount'] = previous_amount

        # Calculate the new amount
        if row['typeop'] == 'BUY':
            new_amount = previous_amount + token_amount
        elif row['typeop'] == 'SELL':
            new_amount = previous_amount - token_amount

        # Set the new amount
        df.at[i, 'new amount'] = new_amount

    return df



df = calculate_amounts(df)


In [ ]:
import numpy as np
def calculate_final_pnl(df):

    

    # Calculate trade_price by dividing source_amount by token_amount
    df['trade_price'] = (df['source_amount'] / df['token_amount']).round(6)

    # Initialize the final_pnl columns
    df['final_pnl %'] = 0.0
    df['final_pnl'] = 0.0

    # Define a small threshold for floating point comparison
    threshold = 1e-8

    for i in range(len(df)):
        row = df.iloc[i]
        mint = row['mint']
        action = row['typeop']
        new_amount = row['new amount']

        # Check if action is 'SELL' and new amount is close to zero within the threshold
        if action == 'SELL' and np.isclose(new_amount, 0, atol=threshold):
            # Find the previous row with the same mint and action 'BUY'
            previous_rows = df[(df['mint'] == mint) & (df['typeop'] == 'BUY') & (df.index < i)]
            if not previous_rows.empty:
                last_row_index = previous_rows.index[-1]
                sol_spent_previous = df.at[last_row_index, 'sol_spent']
                sol_received_current = row['sol_received']
                if sol_spent_previous != 0:
                    final_pnl_percentage = (sol_received_current / sol_spent_previous - 1) * 100  # Convert to percentage
                    final_pnl = sol_received_current - sol_spent_previous  # Calculate absolute PnL
                    df.at[i, 'final_pnl %'] = round(final_pnl_percentage, 5)
                    df.at[i, 'final_pnl'] = round(final_pnl, 5)
    return df


df = calculate_final_pnl(df)

In [ ]:
df.head()

In [ ]:
def accu_pnl_df(df):
    # Filter the DataFrame to only rows where 'final_pnl' is not zero and reset the index
    df_pnl = df[df['final_pnl'] != 0].reset_index(drop=True)

    # Initialize the accumulated_pnl column
    df_pnl['accumulated_pnl'] = 0.0

    # Initialize the accumulated PnL
    accumulated_pnl = 0.0

    for i in range(len(df_pnl)):
        row = df_pnl.iloc[i]
        final_pnl = row['final_pnl']

        # Accumulate the PnL
        accumulated_pnl += final_pnl
        df_pnl.at[i, 'accumulated_pnl'] = round(accumulated_pnl, 5)

    return df_pnl

# Call the function to generate df_pnl
df_pnl = accu_pnl_df(df)


In [ ]:
df_pnl.describe()

In [ ]:
df_pnl.describe()

In [ ]:
# Step 1: Filter the DataFrame for positive values on balance only (to avoid outliers)
filtered_df = df[(df['previous amount'] >= 0) & (df['new amount'] >= 0)]

# Step 2: Calculate the cumulative sums for sol_spent, sol_received, and fees
filtered_df['accumulated_sol_spent'] = filtered_df['sol_spent'].cumsum()
filtered_df['accumulated_sol_received'] = filtered_df['sol_received'].cumsum()
filtered_df['accumulated_fees'] = filtered_df['fee'].cumsum()

# Step 3: Get the last values of accumulated_sol_spent and accumulated_sol_received
total_sol_spent = filtered_df['accumulated_sol_spent'].iloc[-1].round(5)
total_sol_received = filtered_df['accumulated_sol_received'].iloc[-1].round(5)

# Step 4: Calculate the estimated PnL for trades not closed
estimated_pnl = (total_sol_received - total_sol_spent).round(5)

# Step 5: Fees calculation - Get the last value of accumulated_fees
total_fee_spent = (filtered_df['accumulated_fees'].iloc[-1] / 1000000000).round(5)
avg_fee = (total_fee_spent/df['mint'].nunique()).round(5)

# Find the first and last trades
first_trade_timestamp = filtered_df['timestamp'].iloc[0]
last_trade_timestamp = filtered_df['timestamp'].iloc[-1]

avg_trade_size = (filtered_df['accumulated_sol_spent'].iloc[-1]/filtered_df['mint'].nunique()).round(5)

In [ ]:
# Calculate the number of unique values in the 'mint' column
tokens_traded = filtered_df['mint'].nunique() 
trades_closed = df_pnl['mint'].nunique()
trades_open = tokens_traded - trades_closed

# Basic statistics for final_pnl
mean_pnl = df_pnl['final_pnl'].mean()
median_pnl = df_pnl['final_pnl'].median()
std_pnl = df_pnl['final_pnl'].std()
min_pnl = df_pnl['final_pnl'].min()
max_pnl = df_pnl['final_pnl'].max()
percentiles_pnl = df_pnl['final_pnl'].quantile([0.1, 0.25, 0.5, 0.75, 0.9])
total_pnl = df_pnl['final_pnl'].sum()

# Basic statistics for final_pnl %
mean_pnl_percent = df_pnl['final_pnl %'].mean()
median_pnl_percent = df_pnl['final_pnl %'].median()
std_pnl_percent = df_pnl['final_pnl %'].std()
min_pnl_percent = df_pnl['final_pnl %'].min()
max_pnl_percent = df_pnl['final_pnl %'].max()
percentiles_pnl_percent = df_pnl['final_pnl %'].quantile([0.1, 0.25, 0.5, 0.75, 0.9])
total_pnl_percent = df_pnl['final_pnl %'].sum()


# Calc win rate
wins = (df_pnl['final_pnl'] > 0).sum()
losses = (df_pnl['final_pnl'] < 0).sum()
win_rate = wins / (losses + wins) * 100 


In [ ]:
filtered_df.head()

In [ ]:
# drawdown
# min and max entry size 
# min avg and max holding time 


In [ ]:
filtered_df.head()

In [ ]:
# Display the result
print('General wallet perfomance:')

print(f"\nFirst Trade: {first_trade_timestamp}")
print(f"Last Trade: {last_trade_timestamp}")
print(f"Number of unique tokens traded: {tokens_traded}")
print(f"Number of trades closed: {trades_closed}")
print(f"Number of trades open (still has token balance): {trades_open}")
print(f"\nTotal SOL spent (buys): {total_sol_spent}")
print(f"Total SOL received (sells): {total_sol_received}")
print(f"Net SOL (spent - received): {estimated_pnl}")


# Individual closed trades stats
print('\n\nOverview of closed trades:')

print(f'\nWinners: {wins}')
print(f'Losses: {losses}')
print(f"Win Rate: {win_rate:.2f}%")
print(f'Average trade size (sol): {avg_trade_size:.4f}')
print(f"\nMean PnL: {mean_pnl:.4f} or {mean_pnl_percent:.2f}%")
print(f"Standard Deviation of PnL: {std_pnl:.4f} or {std_pnl_percent:.2f}%")
print(f"Minimum PnL: {min_pnl:.4f} or {min_pnl_percent:.2f}%")
print(f"25th Percentile PnL: {percentiles_pnl[0.25]:.4f} or {percentiles_pnl_percent[0.25]:.2f}%")
print(f"50th Percentile (Median) PnL: {percentiles_pnl[0.5]:.4f} or {percentiles_pnl_percent[0.5]:.2f}%")
print(f"75th Percentile PnL: {percentiles_pnl[0.75]:.4f} or {percentiles_pnl_percent[0.75]:.2f}%")
print(f"Maximum PnL: {max_pnl:.4f} or {max_pnl_percent:.2f}%")
print('--------')
print(f"Total PnL: {total_pnl:.4f} or {total_pnl_percent:.2f}%")


#Fees - we have to say that this consider just tx that went through. doesnt consider spam failed txs
print('\n\nFees:')
print(f"Total SOL spent on tx fees: {total_fee_spent:.6f}")
print(f"Avg SOL spent per trade: {avg_fee:.6f}")

In [ ]:
# df.to_excel('output.xlsx', sheet_name='test')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Convert timestamps to only date and hour
df_pnl['timestamp'] = pd.to_datetime(df_pnl['timestamp']).dt.strftime('%Y-%m-%d %H:00')

# Add an initial value of 0 for the accumulated PnL
initial_row = pd.DataFrame({'timestamp': [df_pnl['timestamp'].min()], 'accumulated_pnl': [0]})
df_pnl = pd.concat([initial_row, df_pnl], ignore_index=True)

# Plot the accumulated PnL over time
plt.figure(figsize=(14, 8))

# Plot cumulative PnL
plt.subplot(2, 1, 1)
plt.plot(df_pnl['timestamp'], df_pnl['accumulated_pnl'], marker='o', linestyle='-', color='b')
plt.xlabel('Time')
plt.ylabel('Accumulated PnL')
plt.title('Accumulated PnL Over Time')
plt.xticks(ticks=df_pnl['timestamp'][:], rotation=45, fontsize=8)  
plt.grid()


# Create the new plot (individual trades)
plt.figure(figsize=(12, 4))

# Plot the line for the PnL % over time with an invisible line
plt.plot(df_pnl['timestamp'], df_pnl['final_pnl %'], color='none')

# Overlay the points above y = 0 in green
plt.scatter(df_pnl['timestamp'][df_pnl['final_pnl %'] >= 0], 
            df_pnl['final_pnl %'][df_pnl['final_pnl %'] >= 0], 
            color='green', s=50)

# Overlay the points below y = 0 in red
plt.scatter(df_pnl['timestamp'][df_pnl['final_pnl %'] < 0], 
            df_pnl['final_pnl %'][df_pnl['final_pnl %'] < 0], 
            color='red', s=50)

# Plot settings
plt.title('Closed trades PnL % on Time')
plt.axhline(y=0, color='black', linestyle='--')
plt.xlabel('Timestamp')
plt.ylabel('PnL %')
plt.xticks(ticks=df_pnl['timestamp'][:], rotation=45, fontsize=8)  # Show every label

# Show only horizontal gridlines
plt.grid(True, axis='y')

plt.tight_layout()

# Show legend
plt.legend()

# Show the plot
plt.show()